In [8]:
#!pip install pydub
#!pip install torchaudio==0.7.0
#!pip install librosa==0.8.0
#!pip install torch-audiomentations
#!pip install tensorboard
#!tensorboard --logdir=runs

#!pip install torchaudio
#!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 torchaudio==0.6.0 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch


In [1]:
import os
import sys
import time
import glob
import json
import random
import math
import datetime
import pickle
from collections import defaultdict

In [2]:
os.chdir('C:/Users/pedro/Documents/git/VQ_GAN_music')

In [3]:
sys.path.append('..')

In [4]:
import yaml
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from scipy.io import wavfile

import torch
import torchaudio
torchaudio.set_audio_backend("soundfile")
#torchaudio.set_audio_backend("sox_io")
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler
from torch.utils.tensorboard import SummaryWriter

#import torch_audiomentations
 
import librosa
#import librosa.display
import IPython.display as ipd

C:\Users\pedro\anaconda3\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [5]:
from VQ_GAN_music.datasets.audio_dataset import AudioDatasetNoCond
from VQ_GAN_music.models.vq_vae.vq_vae import VQVAE
from VQ_GAN_music.models.discriminator import MultiDiscriminator
from VQ_GAN_music.models.losses import hinge_loss, vqvae_loss
from VQ_GAN_music.utils.vqvae_trainer import VQVAETrainer

In [6]:
print(torch.version.cuda)

None


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
torch.cuda.is_available()

False

In [10]:
!nvidia-smi

'nvidia-smi' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [11]:
# os.mkdir('Medley-solos-DB')
#zip_path = '/content/drive/MyDrive/IRMAS-TrainingData/IRMAS-TrainingData.zip'
#zip_path = '/content/drive/MyDrive/IRMAS-TrainingData/IRMAS-TrainingData.zip'
#!cp "{zip_path}" .
#!unzip -q IRMAS-TrainingData.zip
#!rm IRMAS-TrainingData.zip

## Dataset

In [12]:
fma_dir =r'C:\Users\pedro\Google Drive (p185770@dac.unicamp.br)\fma_small\*\*.mp3'
#fma_dir = '/content/drive/My Drive/fma_small/fma_small/*/*.mp3'
#fma_val_dir = '/content/drive/My Drive/fma_small/fma_small/029/*.mp3'

In [13]:
with open(r'C:/Users/pedro/Documents/git/VQ_GAN_music/config/fma.yaml') as file:
    hps = yaml.full_load(file)

In [14]:
SAMPLE_RATE = hps['dataset']['sample_rate']
WINDOW_SIZE = int(2**(np.ceil(np.log2(hps['dataset']['win_size']*SAMPLE_RATE)))) # aprox 4 segundos
HOP_LEN = int(2**(np.ceil(np.log2(hps['dataset']['hop_len']*SAMPLE_RATE))))
CONT = 4**(len(hps['model']['vqgan']['vqvae']['ch_mult'])-1)

In [15]:
CONT

256

### Dataset


In [16]:
music_dataset = AudioDatasetNoCond(fma_dir, hps['dataset']['sample_rate'], hps['dataset']['win_size'], hps['dataset']['hop_len'], 1)#, hps['dataset']['b_size'])

In [17]:
music_dataloader = torch.utils.data.DataLoader(music_dataset, batch_size=None, num_workers= hps['dataset']['num_workers'])

In [18]:
data_test = next(iter(music_dataloader))

In [19]:
data_test['inputs'].shape

torch.Size([1, 1, 131072])

In [20]:
ipd.Audio(data_test['inputs'][0], rate = SAMPLE_RATE)

## VQ_VAE

### Models


In [21]:
v_hps = hps['model']['vqgan']['vqvae']
d_hps = hps['model']['vqgan']['disc']
v_num_chs = [v_hps['ch']*mult for mult in v_hps['ch_mult']]
d_num_chs = [d_hps['ch']*mult for mult in d_hps['ch_mult']]

In [22]:
vqvae = VQVAE(v_hps['embed_dim'], v_hps['n_embed'], 1, 1, v_num_chs, v_hps['dilation_depth'], v_hps['attn_indices']).to(device)

In [23]:
discriminator = MultiDiscriminator(d_hps['in_ch'], d_num_chs, 3, WINDOW_SIZE, CONT, n_classes=None).to(device)


In [24]:
#encoder.load_state_dict(torch.load(checkpoint_dir + 'encoder_state.bin'))
#generator.load_state_dict(torch.load(checkpoint_dir + 'generator_state.bin'))

### GAN Trainer

In [25]:
from VQ_GAN_music.utils.vqvae_trainer import VQVAETrainer


In [26]:
gan_trainer = VQVAETrainer(vqvae, discriminator, music_dataloader, vqvae_loss, hinge_loss, hps, device)

In [27]:
samples = gan_trainer.train(1, 'checkpoint_dir', train_gan=True)

Epoch 1/1
----------
torch.Size([1, 128, 32768])
torch.Size([1, 256, 8192])
torch.Size([1, 256, 2048])
torch.Size([1, 512, 512])
torch.Size([1, 256, 2048])
torch.Size([1, 256, 8192])
torch.Size([1, 128, 32768])


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:75] data. DefaultCPUAllocator: not enough memory: you tried to allocate 16777216 bytes. Buy new RAM!